In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/GTA5.zip'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [2]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/MLDL_repo/Cityscapes.zip'
extract_path = '/content/cityscapes'

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Extraction complete!


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/MLDL_repo/step3b')

In [4]:
!pip install -U fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=daa79eb7e294d092790e02edda605439b02da58a4446d6745dd31cba25b320dc
  Stored in directory: /root/.cache/pip/wheels/65/71/95/3b8fde5c65c6e4a806e0867c1651dcc71a1cb2f3430e8f355f
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=6b54977a94f2aa58e2f8bb4eed52daa6bf292d7125855e72fc5d0b906bbd0d5c
  Stored in directory: /root/.cache/pip/wheels/ba/5e/16/6117f8fe7e9c0c161a795e10d94645ebcf301ccbd01f66d8ec
Successfully built fvcore iopath


In [ ]:
import re
import os
import re

def find_latest_checkpoint(checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        print(f"Checkpoint directory {checkpoint_dir} does not exist.")
        return None

    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("bisenet_epoch_") and f.endswith(".pt")]
    if not checkpoints:
        print(f"No checkpoints found in {checkpoint_dir}.")
        return None

    def extract_epoch(fname):
        match = re.search(r"bisenet_epoch_(\d+).pt", fname)
        return int(match.group(1)) if match else -1

    checkpoints.sort(key=extract_epoch, reverse=True)

    latest = os.path.join(checkpoint_dir, checkpoints[0])
    print(f" Found latest checkpoint: {latest}")
    return latest

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

def get_train_transform():
    return A.Compose([
        A.MotionBlur(blur_limit=7, p=0.5),
        A.Resize(720, 1280),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])

def get_val_transform():
    return A.Compose([
        A.Resize(720, 1280),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])


In [ ]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
from torchvision import transforms as T
from PIL import Image
from datasets.gta5_aug import GTA5
from models.bisenet.build_bisenet import BiSeNet
from train import train_one_epoch
from train import validate

def main():

    dataset_root = '/content/dataset/GTA5'

    # Create the full dataset without transforms first
    full_dataset = GTA5(root=dataset_root,transform=None)

    # Then split
    indices = list(range(len(full_dataset)))
    train_indices, val_indices = train_test_split(indices, test_size=0.30, random_state=42)

    # Create two GTA5 datasets with different transforms
    train_dataset = GTA5(root=dataset_root, transform=get_train_transform())
    val_dataset = GTA5(root=dataset_root, transform=get_val_transform())

    # Subset the datasets
    train_dataset = Subset(train_dataset, train_indices)
    val_dataset = Subset(val_dataset, val_indices)
    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Val dataset size: {len(val_dataset)}")

    train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, num_workers=2)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    base_lr = 2.5e-4
    batch_size = 2
    epochs = 50
    context_path = 'resnet18'
    checkpoint_dir = "/content/drive/MyDrive/checkpoints_3b_MotionBlur"

    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    start_epoch = 0
    start_batch = 0

    latest_ckpt = find_latest_checkpoint(checkpoint_dir)
    print("latest_ckpt is:", latest_ckpt)

    if latest_ckpt:
        print(f"Restore from checkpoint: {latest_ckpt}")
        checkpoint = torch.load(latest_ckpt, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        start_batch = 0
        print(f"Picking up from epoch {start_epoch}")
    else:
        print("No checkpoint found, start training from scratch")

    # Check Data and Labels
    for images, labels in train_loader:
        print("Images shape:", images.shape, "dtype:", images.dtype)
        print("Labels shape:", labels.shape, "dtype:", labels.dtype)
        print("Unique labels:", torch.unique(labels))
        break  # Print for the first batch only

    # Training
    for epoch in range(start_epoch, epochs):
        current_start_batch = start_batch if epoch == start_epoch else 0
        train_one_epoch(model, train_loader, optimizer, base_lr, epoch, epochs, device,
                        checkpoint_dir=checkpoint_dir, start_batch=current_start_batch)

    final_model_path = "/content/drive/MyDrive/checkpoints_3b/bisenet_final_3b.pt"
    torch.save(model.state_dict(), final_model_path)
    print(f"Modello finale salvato in: {final_model_path}")

    #Validation
    best_miou = 0.0
    best_miou, miou, per_class_ious = validate(model, val_loader, num_classes, device, best_miou)
    print("\n Results on GTA-V Validation:")
    print(f" - mIoU: {miou:.4f}")
    for idx, iou in enumerate(per_class_ious):
        print(f"Class {idx}: IoU = {iou:.4f}")

if __name__ == "__main__":
    main()


Train dataset size: 1750
Val dataset size: 750
latest_ckpt is: /content/drive/MyDrive/checkpoints_3b/bisenet_epoch_49.pt
Restore from checkpoint: /content/drive/MyDrive/checkpoints_3b/bisenet_epoch_49.pt
Picking up from epoch 50
Images shape: torch.Size([2, 3, 720, 1280]) dtype: torch.float32
Labels shape: torch.Size([2, 720, 1280]) dtype: torch.uint8
Unique labels: tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  13,  14,
         16, 255], dtype=torch.uint8)
Modello finale salvato in: /content/drive/MyDrive/checkpoints_3b/bisenet_final_3b.pt


🔍 Validating: 100%|██████████| 375/375 [01:55<00:00,  3.23it/s]

 Validation mIoU: 0.6536
 New best mIoU found!

 Results on GTA-V Validation:
 - mIoU: 0.6536
Class 0: IoU = 0.9822
Class 1: IoU = 0.8100
Class 2: IoU = 0.8489
Class 3: IoU = 0.6589
Class 4: IoU = 0.3763
Class 5: IoU = 0.7078
Class 6: IoU = 0.5214
Class 7: IoU = 0.5648
Class 8: IoU = 0.8186
Class 9: IoU = 0.7592
Class 10: IoU = 0.9555
Class 11: IoU = 0.4527
Class 12: IoU = 0.4089
Class 13: IoU = 0.8553
Class 14: IoU = 0.7426
Class 15: IoU = 0.7150
Class 16: IoU = 0.7127
Class 17: IoU = 0.4886
Class 18: IoU = 0.0384


In [5]:
import os
import torch
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
from datasets_custom.gta5_aug import GTA5
from datasets_custom.cityscapes import CityScapes
from models.bisenet.build_bisenet import BiSeNet
from datasets_custom.labels import GTA5Labels_TaskCV2017
from train import validate


def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    num_classes = 19
    context_path = 'resnet18'
    batch_size = 2


    model = BiSeNet(num_classes=num_classes, context_path=context_path)
    model = model.to(device)


    final_model_path = "/content/drive/MyDrive/MLDL_repo/step3b/final_models/bisenet_final_3b_MotionBlur.pt"
    model.load_state_dict(torch.load(final_model_path, map_location=device))
    model.eval()
    print(f"Model {final_model_path} has been upload")

    # Prepara il Cityscapes validation loader
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),  # Cityscapes resolution
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ])

    target_transform = transforms.Compose([
        transforms.Resize((512, 1024), interpolation=transforms.InterpolationMode.NEAREST),
        transforms.PILToTensor()
    ])

    dataset_root = '/content/cityscapes/Cityscapes/Cityspaces'

    val_dataset = CityScapes(
    root=dataset_root,
    split='val',
    transform=transform,
    target_transform=target_transform)

    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # results
    best_miou, miou, per_class_ious = validate(model, val_loader, num_classes, device, best_miou=0.0)
    print("\n Validation Results on Cityscape:")
    print(f" - mIoU: {miou:.4f}")
    for idx, label in enumerate(GTA5Labels_TaskCV2017.list_):
        print(f"{label.name:>15}: IoU = {per_class_ious[idx]:.4f}")

if __name__ == "__main__":
    main()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 131MB/s]


Model /content/drive/MyDrive/MLDL_repo/step3b/final_models/bisenet_final_3b_MotionBlur.pt has been upload
Loaded 500 images for split: val


Validating: 100%|██████████| 250/250 [00:56<00:00,  4.39it/s]

 Validation mIoU: 0.1871
 New best mIoU found!

 Validation Results on Cityscape:
 - mIoU: 0.1871
           road: IoU = 0.3777
       sidewalk: IoU = 0.0634
       building: IoU = 0.5743
           wall: IoU = 0.0739
          fence: IoU = 0.0481
           pole: IoU = 0.0001
          light: IoU = 0.0062
           sign: IoU = 0.0020
     vegetation: IoU = 0.7605
        terrain: IoU = 0.0497
            sky: IoU = 0.6551
         person: IoU = 0.3795
          rider: IoU = 0.0683
            car: IoU = 0.4033
          truck: IoU = 0.0508
            bus: IoU = 0.0212
          train: IoU = 0.0046
      motocycle: IoU = 0.0159
        bicycle: IoU = 0.0007
